In [8]:
from ingestion.kivadataloader import KivaDataLoader

m = KivaDataLoader()

print(m.engine.table_names())



[]


In [3]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from config import dbconfig


engine = create_engine(dbconfig.aws_connection_string)
table_names = engine.table_names()
print(table_names)

['partners', 'loan_dates', 'd_date', 'lenders', 'loans', 'loans_lenders']


In [5]:
statement = """SELECT loans.loan_id,original_language,funded_amount,activity_name,sector_name,country_code,borrower_genders,currency,repayment_interval,distribution_model
, loan_dates.* FROM loans
inner join loan_dates on loans.loan_id = loan_dates.loan_id
where status = 'funded' 
and posted_to_raised_days >= 0 and posted_to_raised_days <= 30
"""
#and posted_time_actual >= make_date(2016, 6 ,1) and posted_time_actual <= make_date(2017, 5 ,31)
cleanloans = pd.read_sql_query(statement, engine)

cleanloans.describe()

,loan_id,funded_amount,loan_id,disburse_to_posted_days,posted_to_raised_days,disbursed_to_raised_days
count,1.209612e+06,1.209612e+06,1.209612e+06,1.209592e+06,1.209612e+06,1.209592e+06
mean,7.024412e+05,7.586567e+02,7.024412e+05,1.204523e+01,8.825642e+00,2.129892e+01
std,4.203054e+05,9.977263e+02,4.203054e+05,1.508541e+01,8.199231e+00,1.728952e+01
min,3.080000e+02,2.500000e+01,3.080000e+02,-1.360000e+02,0.000000e+00,-1.100000e+02
25%,3.305378e+05,2.750000e+02,3.305378e+05,5.000000e+00,3.000000e+00,1.200000e+01
50%,6.832365e+05,4.750000e+02,6.832365e+05,1.300000e+01,6.000000e+00,2.300000e+01
75%,1.064144e+06,9.000000e+02,1.064144e+06,2.400000e+01,1.300000e+01,3.300000e+01
max,1.444068e+06,1.000000e+05,1.444068e+06,9.100000e+01,3.000000e+01,1.190000e+02


In [4]:
cleanloans.describe()

,loan_id,funded_amount,loan_id,disburse_to_posted_days,posted_to_raised_days,disbursed_to_raised_days
count,1.783540e+05,178354.000000,1.783540e+05,178354.000000,178354.000000,178354.000000
mean,1.204759e+06,704.752767,1.204759e+06,15.173845,9.896223,25.538379
std,6.216484e+04,1153.332491,6.216484e+04,13.952923,7.845373,15.275543
min,1.092680e+06,25.000000,1.092680e+06,-136.000000,0.000000,-106.000000
25%,1.150955e+06,225.000000,1.150955e+06,7.000000,4.000000,17.000000
50%,1.207906e+06,400.000000,1.207906e+06,17.000000,8.000000,27.000000
75%,1.257332e+06,750.000000,1.257332e+06,26.000000,13.000000,35.000000
max,1.310477e+06,50000.000000,1.310477e+06,32.000000,30.000000,62.000000


In [27]:
cleanloans['num_borrowers_female'] = cleanloans['borrower_genders'].str.count('female')
cleanloans['num_borrowers_male'] = cleanloans['borrower_genders'].str.count('^male') + cleanloans['borrower_genders'].str.count('\, male')
cleanloans['num_borrowers'] = cleanloans['num_borrowers_female']+cleanloans['num_borrowers_male']

cleanloans['num_borrowers_female_pct'] = (cleanloans['num_borrowers_female']*1.00)/cleanloans['num_borrowers']

In [28]:
cleanloans[cleanloans['num_borrowers_female_pct'] == .5]

,loan_id,original_language,funded_amount,activity_name,sector_name,country_code,borrower_genders,currency,repayment_interval,distribution_model,...,posted_time_actual,disburse_time_actual,raised_time_actual,disburse_to_posted_days,posted_to_raised_days,disbursed_to_raised_days,num_borrowers_female,num_borrowers_male,num_borrowers,num_borrowers_female_pct
691,1213663,French,800.0,Retail,Retail,CD,"female, male",USD,monthly,field_partner,...,2016-12-30 04:52:37+00:00,2016-12-21 08:00:00+00:00,2017-03-09 04:40:57+00:00,8.0,68.0,77.0,1.0,1.0,2.0,0.5
737,1208043,French,2000.0,Clothing,Clothing,CD,"male, female",USD,monthly,field_partner,...,2016-12-21 00:30:29+00:00,2016-12-20 08:00:00+00:00,2017-02-26 04:50:03+00:00,0.0,67.0,67.0,1.0,1.0,2.0,0.5
844,1210602,French,1000.0,Clothing Sales,Clothing,CD,"male, female",USD,monthly,field_partner,...,2016-12-26 03:59:29+00:00,2016-12-19 08:00:00+00:00,2017-03-01 01:36:45+00:00,6.0,64.0,71.0,1.0,1.0,2.0,0.5
978,1208837,French,2000.0,Clothing Sales,Clothing,CD,"male, female",USD,monthly,field_partner,...,2016-12-22 07:26:54+00:00,2016-12-17 08:00:00+00:00,2017-02-21 09:54:02+00:00,4.0,61.0,66.0,1.0,1.0,2.0,0.5
1103,1212925,French,800.0,Beverages,Food,CD,"female, male",USD,monthly,field_partner,...,2016-12-29 04:06:55+00:00,2016-12-20 08:00:00+00:00,2017-02-26 09:10:38+00:00,8.0,59.0,68.0,1.0,1.0,2.0,0.5
1159,1209856,French,1500.0,Retail,Retail,CD,"male, female",USD,monthly,field_partner,...,2016-12-23 03:59:57+00:00,2016-12-15 08:00:00+00:00,2017-02-19 05:04:00+00:00,7.0,58.0,65.0,1.0,1.0,2.0,0.5
1187,1204992,French,2000.0,Food,Food,CD,"male, female",USD,monthly,field_partner,...,2016-12-16 04:28:12+00:00,2016-12-13 08:00:00+00:00,2017-02-11 09:21:57+00:00,2.0,57.0,60.0,1.0,1.0,2.0,0.5
1309,939220,English,8325.0,Food,Food,RW,"female, female, female, male, male, male, fema...",RWF,irregular,field_partner,...,2015-08-27 09:54:40+00:00,2015-07-29 07:00:00+00:00,2015-10-23 06:08:44+00:00,29.0,56.0,85.0,6.0,6.0,12.0,0.5
1521,1212309,English,550.0,Personal Housing Expenses,Housing,KH,"female, male",KHR,monthly,field_partner,...,2016-12-29 02:22:56+00:00,2016-12-07 08:00:00+00:00,2017-02-21 06:59:52+00:00,21.0,54.0,75.0,1.0,1.0,2.0,0.5
1653,1185219,English,650.0,Livestock,Agriculture,SL,"male, female",SLL,monthly,field_partner,...,2016-11-15 00:06:46+00:00,2016-11-10 08:00:00+00:00,2017-01-06 03:26:41+00:00,4.0,52.0,56.0,1.0,1.0,2.0,0.5
